In [112]:
import numpy as np
import pandas as pd
import sklearn as sk
import seaborn as sns
import cv2 as cv
import os

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression as LR
from sklearn.ensemble import RandomForestClassifier as RF

import sys
sys.path.append('./Do_s_python_utils')
from resize_imgs import *
from flatten_imgs_np_array import *

## resizing segmented files. 

In [5]:
seg_benign_path = 'C:\\Users\\dohyu\\Documents\\Homework\\stat_542_hw\\stat_542_group\\Segmented\\benign'
seg_malignant_path = 'C:\\Users\\dohyu\\Documents\\Homework\\stat_542_hw\\stat_542_group\\Segmented\\malignant'
seg_benign_path_28 = 'C:\\Users\\dohyu\\Documents\\Homework\\stat_542_hw\\stat_542_group\\Segmented_28\\benign'
seg_malignant_path_28 = 'C:\\Users\\dohyu\\Documents\\Homework\\stat_542_hw\\stat_542_group\\Segmented_28\\malignant'
resize_imgs(seg_benign_path, seg_benign_path_28, (28,28))
resize_imgs(seg_malignant_path, seg_malignant_path_28, (28,28))

## X and y (flattened and splitted)

In [27]:
seg_path_28 = 'C:\\Users\\dohyu\\Documents\\Homework\\stat_542_hw\\stat_542_group\\Segmented_28'
X, y = flatten_imgs_np_array(seg_path_28)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

In [122]:
 np.linspace(start = 100, stop = 500, num = 2)

array([100., 500.])

In [123]:
def perf_summary(y_test,y_pred):
    print(confusion_matrix(y_test,y_pred))
    print(classification_report(y_test,y_pred))

def print_cross_val_score(model, X_train, y_train, num_cv):
    score = cross_val_score(model, X_train, y_train, cv = num_cv )
    avg_acc = np.mean(score)
    std_acc = np.std(score)
    print('avg_acc:{}\nstd_acc:{}'.format(avg_acc, std_acc))
    
def grid_scv(X_train, y_train):
    param_grids = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
              'C':[1, 10]}

    svc_clf = SVC(gamma = 'auto')
    grid_svc = GridSearchCV(svc_clf, param_grids, verbose = 0)
    grid_svc.fit(X_train, y_train)
    return(grid_svc.best_estimator_)

def grid_rf(X_train,y_train):
    n_estimators = [100,500]
    max_depth = [None, 5, 10]
    min_samples_split = [2, 5]
    min_samples_leaf = [1, 2, 4]
    param_grids = {'n_estimators': n_estimators,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf}
    randf_clf = RF(n_jobs = -1)
    grid_RF = GridSearchCV(randf_clf,param_grids, verbose = 0)
    grid_RF.fit(X_train, y_train)
    return(grid_RF.best_estimator_)

### Training logistic with L1 Penalty

In [102]:
lr_clf = LR(penalty = 'l1', solver = 'liblinear')
lr_clf.fit(X_train, y_train)
y_pred = lr_clf.predict(X_test)
perf_summary(y_test,y_pred)

[[20 17]
 [18 20]]
              precision    recall  f1-score   support

           0       0.53      0.54      0.53        37
           1       0.54      0.53      0.53        38

    accuracy                           0.53        75
   macro avg       0.53      0.53      0.53        75
weighted avg       0.53      0.53      0.53        75



In [103]:
score = cross_val_score(lr_clf, X_train, y_train, cv = 10 )
avg_acc = np.mean(score)
std_acc = np.std(score)
print('avg_acc:{}\nstd_acc:{}'.format(avg_acc, std_acc))

avg_acc:0.6300065876152832
std_acc:0.0773935181317182


### Training SVC

In [106]:
svc_clf = grid_scv(X_train, y_train)
svc_clf.fit(X_train, y_train)
y_pred = svc_clf.predict(X_test)
perf_summary(y_test, y_pred)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


[[21 16]
 [17 21]]
              precision    recall  f1-score   support

           0       0.55      0.57      0.56        37
           1       0.57      0.55      0.56        38

    accuracy                           0.56        75
   macro avg       0.56      0.56      0.56        75
weighted avg       0.56      0.56      0.56        75



In [111]:
print_cross_val_score(svc_clf, X_train, y_train, 10)

avg_acc:0.5932641633728589
std_acc:0.08811308064719905


### Training Random Forest

In [125]:
RF_clf = grid_rf(X_train, y_train)
RF_clf.fit(X_train,y_train)
y_pred = RF_clf.predict(X_test)
perf_summary(y_test, y_pred)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


[[27 10]
 [20 18]]
              precision    recall  f1-score   support

           0       0.57      0.73      0.64        37
           1       0.64      0.47      0.55        38

    accuracy                           0.60        75
   macro avg       0.61      0.60      0.59        75
weighted avg       0.61      0.60      0.59        75



In [126]:
print_cross_val_score(RF_clf, X_train,y_train, 10)

avg_acc:0.6873847167325429
std_acc:0.08105584120898149
